# Project Sullivan - Google Drive Training (500GB Dataset)

**Phase 2: Large-Scale Training with Session Persistence**

이 노트북은 Google Drive에 저장된 대용량 데이터셋(500GB)을 활용하여 학습합니다.

---

## 🔑 주요 기능

1. **Google Drive 직접 마운트**: 500GB 데이터셋 접근
2. **세션 유지 (Keep-Alive)**: 90분 비활성 종료 방지
3. **자동 체크포인팅**: 매 에포크마다 Google Drive에 저장
4. **학습 재개**: 세션 종료 후 체크포인트에서 이어서 학습
5. **🆕 SSH 원격 접속**: 로컬 터미널에서 Colab 직접 제어

---

## ⚠️ 사전 요구사항

1. Google Drive에 데이터셋 업로드 완료 (`Sullivan_Dataset/` 폴더)
2. GPU 런타임 설정: Runtime → Change runtime type → GPU (T4)


## ⚙️ Configuration

In [ ]:
# ============================================
# Training Configuration
# ============================================

# GitHub Repository
GITHUB_REPO = 'faransansj/Project_Sullivan'
BRANCH = 'main'

# Google Drive paths
GDRIVE_BASE = '/content/drive/MyDrive'
DATA_DIR = f'{GDRIVE_BASE}/Sullivan_Dataset'
CHECKPOINT_DIR = f'{GDRIVE_BASE}/Sullivan_Checkpoints'
LOG_DIR = f'{GDRIVE_BASE}/Sullivan_Logs'

# Training settings
CONFIG_FILE = 'configs/colab_gdrive_config.yaml'
RESUME_TRAINING = True  # True: 체크포인트에서 재개, False: 새로 시작
QUICK_TEST = False      # True: 10 에포크 테스트, False: 전체 학습

# SSH settings
SSH_PASSWORD = 'sullivan2025'  # SSH 접속 비밀번호

## 🔧 Setup Environment

In [ ]:
# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os

# Verify data directory
if os.path.exists(DATA_DIR):
    print(f'✅ Data directory found: {DATA_DIR}')
    !ls -lh {DATA_DIR}
else:
    print(f'❌ Data directory not found: {DATA_DIR}')
    print('Please upload your dataset to Google Drive first.')

# Create checkpoint and log directories
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)
print(f'\n✅ Checkpoint dir: {CHECKPOINT_DIR}')
print(f'✅ Log dir: {LOG_DIR}')

In [ ]:
# 2. Check GPU availability
import torch
print(f'PyTorch version: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU device: {torch.cuda.get_device_name(0)}')
    print(f'GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB')
else:
    print('⚠️ GPU not available! Please change runtime: Runtime → Change runtime type → GPU')

In [ ]:
# 3. Clone/Update GitHub repository
import os

if not os.path.exists('Project_Sullivan'):
    !git clone https://github.com/{GITHUB_REPO}.git
    %cd Project_Sullivan
else:
    %cd Project_Sullivan
    !git pull origin {BRANCH}

print('\n✅ Repository ready!')

In [ ]:
# 4. Install dependencies
print('📦 Installing dependencies...\n')
!pip install -q torch torchvision torchaudio
!pip install -q pytorch-lightning tensorboard
!pip install -q librosa soundfile
!pip install -q numpy scipy matplotlib seaborn
!pip install -q pyyaml tqdm

print('\n✅ Dependencies installed!')

## 🔌 SSH Remote Access (Optional)

로컬 터미널에서 Colab에 SSH로 접속할 수 있습니다. AI 어시스턴트가 실시간으로 학습을 모니터링하고 문제를 해결할 수 있습니다.

In [ ]:
# SSH Setup via Cloudflare Tunnel
# 이 셀을 실행하면 로컬에서 SSH로 접속할 수 있습니다.

!pip install colab-ssh --quiet

from colab_ssh import launch_ssh_cloudflared

print('='*60)
print('🔌 SSH 터널 설정 중...')
print('='*60)

# Launch SSH with cloudflared tunnel
launch_ssh_cloudflared(password=SSH_PASSWORD)

print('\n' + '='*60)
print('📋 로컬에서 연결하려면:')
print('='*60)
print('1. 위에 출력된 Hostname을 복사하세요')
print('2. 로컬 터미널에서 실행:')
print('   ./scripts/colab_connect.sh save')
print('   (위의 Hostname 입력)')
print('3. SSH 접속:')
print('   ./scripts/colab_connect.sh connect')
print(f'4. 비밀번호: {SSH_PASSWORD}')
print('='*60)

## 🛡️ Session Keep-Alive

세션이 비활성으로 인해 종료되는 것을 방지합니다. 이 셀을 학습 전에 실행하세요.

In [ ]:
# Keep-Alive: 세션 유지를 위한 백그라운드 스레드
import threading
import time
from IPython.display import display, Javascript

def keep_alive_thread():
    """Background thread to keep session alive."""
    while True:
        time.sleep(60)  # 1분마다
        print('.', end='', flush=True)  # 활동 표시

# Start keep-alive thread
keepalive = threading.Thread(target=keep_alive_thread, daemon=True)
keepalive.start()
print('🛡️ Keep-alive thread started. Session will stay active.')

# JavaScript keep-alive (additional safety)
display(Javascript('''
function KeepAlive() {
    console.log("Keep-alive ping");
}
setInterval(KeepAlive, 60000);
console.log("JavaScript keep-alive started");
'''))

## 💾 Checkpoint Management

이전 체크포인트를 확인하고, 학습 재개를 설정합니다.

In [ ]:
# Check existing checkpoints
import glob
import os

checkpoints = glob.glob(f'{CHECKPOINT_DIR}/*.ckpt')

if checkpoints:
    checkpoints.sort(key=os.path.getctime, reverse=True)
    print(f'💾 Found {len(checkpoints)} checkpoint(s):')
    for ckpt in checkpoints[:5]:
        size = os.path.getsize(ckpt) / 1e6
        print(f'   - {os.path.basename(ckpt)} ({size:.1f} MB)')
    
    LATEST_CHECKPOINT = checkpoints[0]
    print(f'\n🔄 Latest checkpoint: {os.path.basename(LATEST_CHECKPOINT)}')
    
    if RESUME_TRAINING:
        print('✅ Training will RESUME from latest checkpoint.')
    else:
        print('⚠️ RESUME_TRAINING=False: Training will start fresh (ignoring checkpoints).')
else:
    print('📭 No checkpoints found. Training will start fresh.')
    LATEST_CHECKPOINT = None

## 🏋️ Model Training

In [ ]:
# Start training with checkpoint resume support
import subprocess
import sys

print(f'🚀 Starting Transformer training...')
print(f'   Config: {CONFIG_FILE}')
print(f'   Data: {DATA_DIR}')
print(f'   Checkpoints: {CHECKPOINT_DIR}')
print(f'   Resume: {RESUME_TRAINING}')
print(f'   Quick test: {QUICK_TEST}')
print()

# Build command
cmd = [
    sys.executable, 'scripts/train_transformer.py',
    '--config', CONFIG_FILE,
    '--gpus', '1',
    '--data-dir', DATA_DIR,
    '--checkpoint-dir', CHECKPOINT_DIR,
    '--log-dir', LOG_DIR,
]

if RESUME_TRAINING and LATEST_CHECKPOINT:
    cmd.extend(['--resume-from', LATEST_CHECKPOINT])

if QUICK_TEST:
    cmd.extend(['--max-epochs', '10'])

# Run training
print(f'Command: {" ".join(cmd)}')
print('=' * 60)

process = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True,
    bufsize=1
)

for line in iter(process.stdout.readline, ''):
    print(line, end='')

process.wait()

if process.returncode == 0:
    print('\n✅ Training completed successfully!')
else:
    print(f'\n❌ Training failed with return code: {process.returncode}')

## 📊 Monitor Training with TensorBoard

In [ ]:
# Launch TensorBoard
%load_ext tensorboard
%tensorboard --logdir {LOG_DIR}

## 📈 View Training Results

In [ ]:
# Check final results
import glob
import os
import pandas as pd

# Find latest checkpoint
checkpoints = glob.glob(f'{CHECKPOINT_DIR}/*.ckpt')
if checkpoints:
    checkpoints.sort(key=os.path.getctime, reverse=True)
    print(f'💾 Final checkpoints ({len(checkpoints)} total):')
    for ckpt in checkpoints[:3]:
        size = os.path.getsize(ckpt) / 1e6
        print(f'   - {os.path.basename(ckpt)} ({size:.1f} MB)')

# Check metrics if available
metrics_files = glob.glob(f'{LOG_DIR}/**/metrics.csv', recursive=True)
if metrics_files:
    latest_metrics = max(metrics_files, key=os.path.getctime)
    print(f'\n📊 Training Metrics: {latest_metrics}')
    df = pd.read_csv(latest_metrics)
    print(df.tail(10))

---

## 📝 Notes

### Session Persistence
- **Keep-Alive**: 백그라운드 스레드가 세션 유지
- **Auto-Checkpoint**: 매 에포크마다 Google Drive에 저장
- **Resume**: 세션 종료 후 다시 실행하면 자동으로 이어서 학습

### SSH Remote Access
- **SSH Setup 셀 실행 후** 로컬에서 `./scripts/colab_connect.sh save` 로 연결정보 저장
- `./scripts/colab_connect.sh connect` 로 SSH 접속
- `./scripts/colab_connect.sh run 'nvidia-smi'` 로 원격 명령 실행

### Troubleshooting

**GPU OOM:**
- `configs/colab_gdrive_config.yaml`에서 `batch_size` 줄이기
- `precision: 16` (Mixed precision) 활성화 확인

**세션 종료됨:**
- 이 노트북 다시 실행 (RESUME_TRAINING=True)
- 마지막 체크포인트에서 자동 재개

**데이터 로딩 느림:**
- Streaming DataLoader 사용 확인
- `num_workers` 조정

---

**Last Updated**: 2025-12-23
**Project**: Sullivan - Acoustic-to-Articulatory Inversion
**Setup**: Google Drive 500GB + Colab Free Tier + SSH Remote Access